exp1 :- (BOW/Tfidf) , unigram ,bigram ,tfidf
exp2 :- max_feature(hp tune)
exp3 :- Imbalance learning ==> trying out various sampling techniques to tackle the issue [undersampling, oversampling ,SMOTE , adasyn,SMOTE ENN, class weights]
exp4 :- Try out multiple Algo
exp5 :- select the best model and do the detailed HP tune.

###### finding which setting for FE is best -> tfidf or bow --> with ngram value(uni,bi,tri)


In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 996.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 601.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [9]:
!aws configure

In [3]:
import mlflow
# set tracking uri
mlflow.set_tracking_uri("YOUR TRACKING URI")

In [4]:
# create / set an experiment
mlflow.set_experiment("EXP 2 - BOW v/s Tfidf")

2024/11/19 16:36:45 INFO mlflow.tracking.fluent: Experiment with name 'EXP 2 - BOW v/s Tfidf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://interview-mlflow-bucket/173426427286385857', creation_time=1732034205500, experiment_id='173426427286385857', last_update_time=1732034205500, lifecycle_stage='active', name='EXP 2 - BOW v/s Tfidf', tags={}>

In [ ]:
#!pip install mlflow

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/ashishshukla888/influencer_sentiment_analysis/refs/heads/master/data.csv').dropna(subset=['clean_comment'])
df.shape

(37149, 2)

In [8]:
# S1 :- Func to run the exp

def run_experiment(vectorizer_type,ngram_range,vectorizer_max_features,vectorizer_name):

  # S2 :- Vectorization
  if vectorizer_type == "BoW":
    vectorizer = CountVectorizer(ngram_range = ngram_range, max_features = vectorizer_max_features )
  else:
    vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features = vectorizer_max_features )

  # S3 :- split data
  X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42,stratify=df['category'])

  X_train = vectorizer.fit_transform(X_train)
  X_test = vectorizer.transform(X_test)

  # S4 :- Define & train a RF model
  with mlflow.start_run() as run:

    # set tags for exp & run
    mlflow.set_tag("mlflow.runName",f"{vectorizer_name}_{ngram_range}_RandomForest")
    mlflow.set_tag("experiment_type","feature_engineering")
    mlflow.set_tag("model_type","RandomForestClassifier")

    # Add a description
    mlflow.set_tag("desciption",f"RandomForest with {vectorizer_name}, ngram_range={ngram_range},max_feature = {vectorizer_max_features}")

    # Log vectorizer parameters
    mlflow.log_param("vectorizer_type",vectorizer_type)
    mlflow.log_param("ngram_range",ngram_range)
    mlflow.log_param("vectorizer_max_features",vectorizer_max_features)

    # Log RF params
    n_estimators = 200
    max_depth = 15

    mlflow.log_param("n_estimators",n_estimators)
    mlflow.log_param("max_depth",max_depth)

    # Train model
    model = RandomForestClassifier(n_estimators=n_estimators , max_depth = max_depth , random_state = 42)
    model.fit(X_train,y_train)

    # S5 :- Predict & log metric
    y_pred = model.predict(X_test)

    # Log accuracy
    accuracy = accuracy_score(y_test,y_pred)
    mlflow.log_metric("accuracy",accuracy)

    # Log classification report
    classification_rep = classification_report(y_test, y_pred , output_dict = True)
    for label,metrics in classification_rep.items():
      if isinstance(metrics,dict):
        for metric, value in metrics.items():
          mlflow.log_metric(f"{label}_{metric}",value)

    # log CMatrix
    conf_matrix = confusion_matrix(y_test,y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(conf_matrix,annot=True,fmt="d",cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    # Log the model
    mlflow.sklearn.log_model(model,f"random_forest_model_{vectorizer_name}_{ngram_range}")

# S6 :- Run experiments for BOW & TF-IDF with diff n-grams
ngram_ranges = [(1,1),(1,2),(1,3)]
max_features = 5000 # Example max feature size

for ngram_range in ngram_ranges:
  # BoW experiment
  run_experiment("BoW",ngram_range,max_features,vectorizer_name="BoW")

  # TF-IDF Exp
  run_experiment("TF-IDF",ngram_range,max_features,vectorizer_name="TF-IDF")

2024/11/19 16:43:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857/runs/86ed3cc9796d4dd1ba8ac196c9625ee9
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857


2024/11/19 16:43:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857/runs/9b5727190f4d48b59470e6bf1932f19d
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857


2024/11/19 16:43:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857/runs/7213a7a53ce14f31b7b505c026240ad8
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857


2024/11/19 16:44:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857/runs/78b5b74803124d1bb14b86e0f39565fb
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857


2024/11/19 16:44:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857/runs/17395fb4377b4406b45b9f403c1f5fa1
🧪 View experiment at: http://ec2-35-174-3-91.compute-1.amazonaws.com:5000/#/experiments/173426427286385857
